In [ ]:
#!pip install -q ctransformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00


In [1]:
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML", model_file = 'llama-2-7b-chat.ggmlv3.q2_K.bin', callbacks=[StreamingStdOutCallbackHandler()],temperature=0)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\mm0450.MAVEN\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mm0450.MAVEN\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
template = """
[INST] <<SYS>>
You are a accurate data annotator of Recipe and respond single word without any reasoning. Using the given ingredients of a recipe, You need to just return whether it is 'NonVeg' or 'Veg' or 'Vegan'. Ingredients: 
<</SYS>>
{ingredients}[/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["ingredients"])

In [14]:
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

In [15]:
ingredients = "['1 c. firmly packed brown sugar', '1/2 c. evaporated milk', '1/2 tsp. vanilla', '1/2 c. broken nuts (pecans)', '2 Tbsp. butter or margarine', '3 1/2 c. bite size shredded rice biscuits']"

In [16]:
response = llm_chain.run(ingredients)



> Entering new LLMChain chain...
Prompt after formatting:

[INST] <<SYS>>
You are a accurate data annotator of Recipe and respond single word without any reasoning. Using the given ingredients of a recipe, You need to just return MAXIMUM ONE WORD and nothing else whether it is 'NonVeg' or 'Veg' or 'Vegan'. Ingredients: 
<</SYS>>
['1 c. firmly packed brown sugar', '1/2 c. evaporated milk', '1/2 tsp. vanilla', '1/2 c. broken nuts (pecans)', '2 Tbsp. butter or margarine', '3 1/2 c. bite size shredded rice biscuits'][/INST]


MAXIMUM ONE WORD: Veg
> Finished chain.


Classification Testing on Data

In [25]:
import pandas as pd

In [26]:
df = pd.read_csv("recipe_top_1000_rows.csv")

In [27]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1000 non-null   int64 
 1   title        1000 non-null   object
 2   ingredients  1000 non-null   object
 3   directions   1000 non-null   object
 4   link         1000 non-null   object
 5   source       1000 non-null   object
 6   NER          1000 non-null   object
 7   site         1000 non-null   object
dtypes: int64(1), object(7)
memory usage: 62.6+ KB


,Unnamed: 0,title,ingredients,directions,link,source,NER,site
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com


In [28]:
df = df.head(20)

In [29]:
df['classification'] = df.ingredients.apply(lambda x: llm_chain.run(x))

The ingredients in the recipe indicate that it is a vegan recipe, therefore the classification of the ingredients would be "Veg".The ingredients in the recipe are:

* Chipped beef (NonVeg)
* Boned chicken breasts (NonVeg)
* Cream of mushroom soup (NonVeg)
* Sour cream (NonVeg)

Therefore, the strict classification of the ingredients in this recipe is: NonVeg.The ingredients in the recipe are:

* Frozen corn (NonVeg)
* Cream cheese (Veg)
* Butter (Veg)
* Garlic powder (NonVeg)
* Salt (NonVeg)
* Pepper (NonVeg)

Therefore, the classification of this recipe is "Vegan".VegThe ingredients in the recipe are:

* Peanut butter
* Graham cracker crumbs
* Melted butter
* Powdered sugar (3 1/2 cups)
* Chocolate chips (1 lb)

Based on these ingredients, I would classify this recipe as a Vegan recipe.The ingredients in the recipe indicate that it is a Vegan dish. Therefore, the classification of the dish is "Veg".The ingredients in the recipe indicate that it is a vegan recipe, therefore the strict 

In [30]:
df.head(20)

,Unnamed: 0,title,ingredients,directions,link,source,NER,site,classification
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com,The ingredients in the recipe indicate that it...
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com,The ingredients in the recipe are:\n\n* Chippe...
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com,The ingredients in the recipe are:\n\n* Frozen...
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com,Veg
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com,The ingredients in the recipe are:\n\n* Peanut...
5,5,Cheeseburger Potato Soup,"[""6 baking potatoes"", ""1 lb. of extra lean gro...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"[""sour cream"", ""bacon"", ""pepper"", ""extra lean ...",www.cookbooks.com,The ingredients in the recipe indicate that it...
6,6,Rhubarb Coffee Cake,"[""1 1/2 c. sugar"", ""1/2 c. butter"", ""1 egg"", ""...","[""Cream sugar and butter."", ""Add egg and beat ...",www.cookbooks.com/Recipe-Details.aspx?id=210288,Gathered,"[""buttermilk"", ""egg"", ""sugar"", ""vanilla"", ""sod...",www.cookbooks.com,The ingredients in the recipe indicate that it...
7,7,Scalloped Corn,"[""1 can cream-style corn"", ""1 can whole kernel...","[""Mix together both cans of corn, crackers, eg...",www.cookbooks.com/Recipe-Details.aspx?id=876969,Gathered,"[""egg"", ""pepper"", ""crackers"", ""cream-style cor...",www.cookbooks.com,The ingredients in the recipe do not suggest a...
8,8,Nolan'S Pepper Steak,"[""1 1/2 lb. round steak (1-inch thick), cut in...","[""Roll steak strips in flour."", ""Brown in skil...",www.cookbooks.com/Recipe-Details.aspx?id=375254,Gathered,"[""oil"", ""tomatoes"", ""green peppers"", ""water"", ...",www.cookbooks.com,\nReturning a single word for the given ingred...
9,9,Millionaire Pie,"[""1 large container Cool Whip"", ""1 large can c...","[""Empty Cool Whip into a bowl."", ""Drain juice ...",www.cookbooks.com/Recipe-Details.aspx?id=794547,Gathered,"[""condensed milk"", ""lemons"", ""graham cracker c...",www.cookbooks.com,The ingredients in the recipe do not indicate ...
